In [135]:

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import *


In [215]:
#import summer_stuff.reinforcement_learning.env

import env
reload(env)
import agent
reload(agent)
import sensors
reload(sensors)
import brain
reload(brain)

<module 'brain' from 'brain.pyc'>

In [216]:
b = env.env(600,600, show=False)

In [217]:
b.load_from_yaml('data.yml')

In [218]:
a = agent.agent(x=15,y=15, ray_dist=50, ray_count=4, ray_width=70, show=True)

In [219]:
b.add_agent(a)

In [220]:
b.draw()

In [222]:
import time

start = time.time()
num_tries = 100
for i in range(num_tries):
    observations = a.observe()
    for t in range(100):
        action = a.brain.decide(observations)
        observations, reward, done, info = a.step(action)
   
        if done:
            break

    a.reset()
end  = time.time()
print (end-start)

20.4909369946


In [110]:
#for i_episode in range(20):
#    observation = env.reset()
#    for t in range(100):
#        env.render()
#        print(observation)
#        action = env.action_space.sample()
#        observation, reward, done, info = env.step(action)
#        if done:
#            print("Episode finished after {} timesteps".format(t+1))
#            break

In [ ]:
import pygame
import sys


pygame.init()

In [ ]:
screen = pygame.display.set_mode((640,480))

In [ ]:

pygame.display.update()
screen.fill(black)

In [ ]:
red = (255,0,0)
green = (0,255,0)
blue = (0,0,255)
darkBlue = (0,0,128)
white = (255,255,255)
black = (0,0,0)
pink = (255,200,200)

In [ ]:

pygame.draw.circle(screen, red,  (50,100), 25, 1)


In [ ]:
pygame.quit(); sys.exit();

In [ ]:
while (True):

   # check for quit events
   for event in pygame.event.get():
        if event.type == pygame.QUIT:
             pygame.quit(); sys.exit();

   # erase the screen
   screen.fill(white)

   # draw the updated picture

   updatePoints(points)  # changes the location of the points
   pygame.draw.lines(screen,black,false,points,1)  # redraw the points

   # update the screen
   pygame.display.update()

In [ ]:
int((34.4,56))

In [41]:
def fun(a, z=None):

    b = z or 'a'
    print b

In [43]:
fun(2, z=(1,2,4))

(1, 2, 4)


In [151]:
[1,2,3,4][1::2]

[2, 4]